Faster than Light Speed Quantum Teleportation for Bit Communication Between Alice and Bob.

In [ ]:
# Code for Creating A 8 Qubit Entangled State - But it is not used elsewhere in the code - Also a Unitary Circuit of Mapping from input Qubit Ground State to 
# This entanglement AME state ca be found anf this code can be extended to 2**40 Qubits , the rough limit of Classical Compute.

import torch
import torch.nn as nn
import torch.optim as optim
from itertools import combinations
import time

class OctoQubitEntangledState(nn.Module):
    def __init__(self):
        super().__init__()
        self.state_dim = 2**8  
        self.params = nn.Parameter(torch.randn(2*self.state_dim)) 
        
    def forward(self):
        real_part = self.params[:self.state_dim]
        imag_part = self.params[self.state_dim:]
        state = torch.complex(real_part, imag_part)
        return state / torch.norm(state)

    def compute_reduced_density(self, state, kept_qubits):
        """Efficient reduced density matrix calculation with stabilization"""
        state_tensor = state.reshape([2]*8)
        kept = sorted(kept_qubits)
        trace_dims = sorted(set(range(8)) - set(kept))
        
        permuted = state_tensor.permute(kept + trace_dims)
        d_keep = 2**len(kept)
        reshaped = permuted.reshape(d_keep, -1)
        rho = reshaped @ reshaped.conj().T
        return rho + 1e-8 * torch.eye(d_keep, dtype=torch.complex64)

    def compute_reduced_densitytrace(self, state, kept_qubits):
        """Efficient reduced density matrix calculation with stabilization"""
        state_tensor = state.reshape([2]*8)
        kept = sorted(kept_qubits)
        trace_dims = sorted(set(range(8)) - set(kept))
        
        permuted = state_tensor.permute(kept + trace_dims)
        d_keep = 2**len(kept)
        reshaped = permuted.reshape(d_keep, -1)
        rho = reshaped @ reshaped.conj().T
        purity = torch.trace(rho@rho).real
        return purity

    def entanglement_criteria(self, state):
        loss = 0.0
        max_entropy = {1: 1.0, 2: 2.0, 3: 3.0, 4: 4.0}
        reg_strength = 1e-8  # Regularization parameter
        
        for k in [1, 2, 3, 4]:
            for kept in combinations(range(8), k):
                rho = self.compute_reduced_density(state, kept)
                d = 2**k
                identity = torch.eye(d, dtype=torch.complex64) / d
                
                # Criterion 1: State similarity
                loss += torch.norm(rho - identity, p='fro')**2
                
                # Criterion 2: Entanglement entropy with stabilization
                try:
                    eigvals = torch.linalg.eigvalsh(rho)
                except torch.linalg.LinAlgError:
                    eigvals = torch.ones(d) / d 
                
                eigvals = torch.clamp(eigvals, min=reg_strength, max=1.0)
                entropy = -torch.sum(eigvals * torch.log2(eigvals + reg_strength))
                loss += (entropy.real - max_entropy[k])**2
                
                # Criterion 3: Purity constraint
                purity = torch.trace(rho @ rho).real
                loss += (purity - 1/d)**2

        return loss

class StabilizedLRScheduler:
    def __init__(self, optimizer, factor=0.5, patience=3, min_lr=1e-6):
        self.optimizer = optimizer
        self.factor = factor
        self.patience = patience
        self.min_lr = min_lr
        self.best_loss = float('inf')
        self.counter = 0
        self.current_lr = optimizer.param_groups[0]['lr']

    def step(self, current_loss):
        if current_loss < self.best_loss:
            self.best_loss = current_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self._reduce_lr()

    def _reduce_lr(self):
        new_lr = max(self.current_lr * self.factor, self.min_lr)
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = new_lr
        self.current_lr = new_lr
        self.counter = 0

    def get_last_lr(self):
        return [self.current_lr]

def train_8qubit_ame():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = OctoQubitEntangledState().to(device)
    optimizer = optim.AdamW(model.parameters(), lr=0.002, weight_decay=1e-5)
    scheduler = StabilizedLRScheduler(optimizer)
    
    best_loss = float('inf')
    patience = 500
    no_improve = 0
    
    print("Training 8-qubit AME state...")
    start_time = time.time()
    
    try:
        for epoch in range(10000):
            optimizer.zero_grad()
            state = model()
            loss = model.entanglement_criteria(state)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step(loss.item())
            if loss.item() < best_loss:
                best_loss = loss.item()
                no_improve = 0
            else:
                no_improve += 1
                
            if epoch % 50 == 0:
                print(f"Epoch {epoch:4d} | Loss: {loss.item():.4f} | LR: {scheduler.current_lr:.2e}")
                
            if no_improve >= patience:
                print(f"Early stopping at epoch {epoch}")
                break
                
    except KeyboardInterrupt:
        print("Training interrupted by user")
    
    print(f"Training completed in {time.time()-start_time:.2f}s")
    return model()

def verify_entanglement(state, model):
    print("\nVerification metrics:")
    model.eval()
    with torch.no_grad():
        for k in [1, 4]:
            combs = list(combinations(range(8), k))
            for kept in combs: 
                rho = model.compute_reduced_density(state, kept)
                d = 2**k
                
                eigvals = torch.linalg.eigvalsh(rho).real
                entropy = -torch.sum(eigvals * torch.log2(eigvals + 1e-12))
                purity = torch.trace(rho @ rho).real
                
                print(f"{k}-qubit reduction {kept}:")
                print(f"  Entropy: {entropy:.4f} (target {k})")
                print(f"  Purity: {purity:.4f} (target {1/d:.3f})")
                print(f"  Eigenvalues: {eigvals.cpu().numpy().round(4)}")

if __name__ == "__main__":
    final_state = train_8qubit_ame()
    verify_entanglement(final_state, OctoQubitEntangledState())

Training 8-qubit AME state...
Epoch    0 | Loss: 45.5321 | LR: 2.00e-03
Training interrupted by user
Training completed in 5.70s

Verification metrics:
1-qubit reduction (0,):
  Entropy: 0.9908 (target 1)
  Purity: 0.5063 (target 0.500)
  Eigenvalues: [0.4437 0.5563]
1-qubit reduction (1,):
  Entropy: 0.9950 (target 1)
  Purity: 0.5035 (target 0.500)
  Eigenvalues: [0.4582 0.5418]
1-qubit reduction (2,):
  Entropy: 0.9865 (target 1)
  Purity: 0.5093 (target 0.500)
  Eigenvalues: [0.4317 0.5683]
1-qubit reduction (3,):
  Entropy: 0.9965 (target 1)
  Purity: 0.5024 (target 0.500)
  Eigenvalues: [0.465 0.535]
1-qubit reduction (4,):
  Entropy: 0.9954 (target 1)
  Purity: 0.5032 (target 0.500)
  Eigenvalues: [0.4601 0.5399]
1-qubit reduction (5,):
  Entropy: 0.9905 (target 1)
  Purity: 0.5066 (target 0.500)
  Eigenvalues: [0.4426 0.5574]
1-qubit reduction (6,):
  Entropy: 0.9924 (target 1)
  Purity: 0.5052 (target 0.500)
  Eigenvalues: [0.4489 0.5511]
1-qubit reduction (7,):
  Entropy: 0.9

In [1]:
# Code For Finding the Two UNitary Matrices of Alice for zero and one Bit Communication , Unitary Matrix of Classification of Bob , the Partially Entangled
# State that Bob and Alice Shares , Partial Entanglement between Alice's Output State of here Eight Qubits by using Backpropogation Algorithm to minimize the 
# RMSE loss between the ouptut states i.e. the Bob's States after he applies his unitary of Clasification to classify the input state that Alice Sends , which can 
# be any one of the 256 output states of class 0 or 256 Output states of class 1. Also entanglement Constriants are added on the input state that Alice Shares
# with Bob , and also the top four and the bottom four qubits of the Alice , to further entangle the Alice's state with her part of the entangled qubits that
# she shares with Bob. Here No cloning is not violated as one can always use the same unitary circuit to create a approximate replica , not clone it , but produce it
# again , so same entangled state pairs are shared in 1000's or lakhs for entire communication priorly when ALice and BOb are together.

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class QuantumUnitary(nn.Module):
    def __init__(self, dim=256, dim_separating=16):
        super(QuantumUnitary, self).__init__()
        self.state1 = nn.Parameter(torch.randn(16, dtype=torch.cfloat).contiguous()) # Input state that Alice wants to send to BOB for communication in Circuit 1 of Alice
        self.state2 = nn.Parameter(torch.randn(16, dtype=torch.cfloat).contiguous()) # Input state that Alice wants to send to BOB for communication in Circuit 2 of Alice
        self.finalstate = nn.Parameter(torch.randn(256, dtype=torch.cfloat).contiguous()) # Entangled State that ALice shares with Bob
        self.U1 = nn.Parameter(torch.randn(dim, dim, dtype=torch.cfloat).contiguous()) # Alice 's UNitary Circuit that she can apply on her eight qubits to teleport one of the 256 states that she gets in superposition after application of this unitary to BOb, but that any quantum states sent belong to Class 0.
        self.U2 = nn.Parameter(torch.randn(dim, dim, dtype=torch.cfloat).contiguous()) # Alice 's UNitary Circuit that she can apply on her eight qubits to teleport one of the 256 states that she gets in superposition after application of this unitary to BOb, but that any quantum states sent belong to Class 1.
        self.Useparating = nn.Parameter(torch.randn(dim_separating, dim_separating, dtype=torch.cfloat).contiguous()) # Bob Seperating Unitary to classify into one of the two classes.
        self._enforce_constraints()
        
    def _enforce_constraints(self):
        with torch.no_grad():
            self.state1.data = F.normalize(self.state1.data.contiguous(), p=2, dim=0)
            self.state2.data = F.normalize(self.state2.data.contiguous(), p=2, dim=0)
            self.finalstate.data = F.normalize(self.finalstate.data.contiguous(), p=2, dim=0)
            for param in [self.U1, self.U2, self.Useparating]:
                U, _ = torch.linalg.qr(param.data.contiguous())
                param.data = U.contiguous()

    def forward(self, final_state):
        self._enforce_constraints()
        state1 = torch.kron(self.state1, self.finalstate)
        state2 = torch.kron(self.state2, self.finalstate)
        state1 = F.normalize(state1, p=2, dim=0)
        state2 = F.normalize(state2, p=2, dim=0)
        I = torch.eye(16, dtype=torch.cfloat)
        U_total1 = torch.kron(self.U1, I)
        e = (U_total1 @ state1).reshape(256, 16) # Measurement at Alice Side for first Circuit , which acts as input states of class 0 of training the Unitary of Classification
        # at Bob
        U_total2 = torch.kron(self.U2, I)
        f = (U_total2 @ state2).reshape(256, 16) # Measurement at Alice's Side for Secoend Circuit , which acts as input states of training 0f class 1 of the Unitary of Classification
        # at Bob

        g = torch.zeros(256, 16, dtype=torch.cfloat)
        h = torch.zeros(256, 16, dtype=torch.cfloat)
        for i in range(256):
            g[i] = self.Useparating @ e[i]
            h[i] = self.Useparating @ f[i]
            
        return F.normalize(g, p=2, dim=1), F.normalize(h, p=2, dim=1)


def enhanced_entanglement_loss(finalstate):
    """Enforce entanglement via multiple criteria: Schmidt rank, entropy, purity, and negativity"""
    state_matrix = finalstate.reshape(16, 16)

    _, S, _ = torch.linalg.svd(state_matrix)
    
    # 1. Schmidt Rank Loss - encourage multiple non-zero singular values (>1)
    # Higher Schmidt rank means more entanglement
    schmidt_rank = torch.sum(S > 1e-6).float()
    schmidt_rank_loss = torch.relu(2.0 - schmidt_rank)  # We want at least 2 significant singular values
    
    # 2. Von Neumann Entropy - higher entropy means more entanglement
    # Calculate normalized singular values/Eigen Values (probabilities)
    S_squared = S**2 / torch.sum(S**2)
    entropy = -torch.sum(S_squared * torch.log2(S_squared + 1e-12))
    entropy_loss = torch.relu(3.0 - entropy)  # We want Entanglement entropy > 1
    
    # 3. Subsystem Purity Loss - encourage low purity in reduced state
    # For entangled states, reduced density matrix has purity < 1
    # Calculate reduced density matrix by partial trace
    rho = torch.outer(finalstate, finalstate.conj())  # Full density matrix
    rho_reduced = partial_trace(rho, 16, 16)  # Partial trace over second subsystem
    reduced_purity = torch.trace(rho_reduced @ rho_reduced).real
    purity_loss = reduced_purity  # Minimize purity of reduced state (more entangled = lower purity)
    
    # 4. Negativity - negative eigenvalues after partial transpose indicate entanglement
    rho_pt = partial_transpose(rho, 16, 16)  # Partial transpose
    eigenvalues = torch.linalg.eigvalsh(rho_pt)
    negativity = -torch.sum(torch.minimum(eigenvalues, torch.tensor(0.0)))
    negativity_loss = torch.relu(0.1 - negativity)  # Encourage negativity > 0.1


    print(f'Schmidt Rank: {schmidt_rank.item()}, Entropy: {entropy.item()}, Reduced Purity: {reduced_purity.item()}, Negativity: {negativity.item()}')
    
    # Combine all entanglement measures with appropriate weights
    total_entanglement_loss = (
        0.5 * schmidt_rank_loss + 
        0.5 * entropy_loss + 
        0.3 * purity_loss + 
        0.2 * negativity_loss
    )
    
    return total_entanglement_loss

def enhanced_entanglement_lossnew(newfinalstate):
    """Enforce entanglement via multiple criteria: Schmidt rank, entropy, purity, and negativity"""
    newfinalstate = F.normalize(newfinalstate,p=2,dim=0)
    state_matrix = newfinalstate.reshape(256,16)
    
    # Perform Schmidt decomposition
    _, S, _ = torch.linalg.svd(state_matrix)
    
    # 1. Schmidt Rank Loss - encourage multiple non-zero singular values (>1)
    # Higher Schmidt rank means more entanglement
    schmidt_rank = torch.sum(S > 1e-6).float()
    schmidt_rank_loss = torch.relu(2.0 - schmidt_rank)  # We want at least 2 significant singular values
    
    # 2. Von Neumann Entropy - higher entropy means more entanglement
    # Calculate normalized singular values (probabilities)
    S_squared = S**2 / torch.sum(S**2)
    entropy = -torch.sum(S_squared * torch.log2(S_squared + 1e-12))
    entropy_loss = torch.relu(3.0 - entropy)  # We want entropy > 1
    
    # 3. Subsystem Purity Loss - encourage low purity in reduced state
    # For entangled states, reduced density matrix has purity < 1
    # Calculate reduced density matrix by partial trace
    rho = torch.outer(newfinalstate, newfinalstate.conj())  # Full density matrix
    rho_reduced = partial_trace(rho, 256,16)  # Partial trace over second subsystem
    reduced_purity = torch.trace(rho_reduced @ rho_reduced).real
    purity_loss = reduced_purity  # Minimize purity of reduced state (more entangled = lower purity)
    
    # 4. Negativity - negative eigenvalues after partial transpose indicate entanglement
    rho_pt = partial_transpose(rho, 256, 16)  # Partial transpose
    eigenvalues = torch.linalg.eigvalsh(rho_pt)
    negativity = -torch.sum(torch.minimum(eigenvalues, torch.tensor(0.0)))
    negativity_loss = torch.relu(0.1 - negativity)  # Encourage negativity > 0.1

    print(f'Schmidt Rank: {schmidt_rank.item()}, Entropy: {entropy.item()}, Reduced Purity: {reduced_purity.item()}, Negativity: {negativity.item()}')
    
    # Combine all entanglement measures with appropriate weights
    total_entanglement_loss = (
        0.5 * schmidt_rank_loss + 
        0.5 * entropy_loss + 
        0.3 * purity_loss + 
        0.2 * negativity_loss
    )
    
    return total_entanglement_loss

def partial_trace(rho, dim_A, dim_B):
    """Calculate partial trace over system B"""
    rho_reshaped = rho.reshape(dim_A, dim_B, dim_A, dim_B)
    rho_A = torch.zeros((dim_A, dim_A), dtype=torch.cfloat, device=rho.device)
    
    for i in range(dim_B):
        rho_A += rho_reshaped[:, i, :, i]
    
    return rho_A

def partial_transpose(rho, dim_A, dim_B):
    """Calculate partial transpose with respect to system B"""
    rho_reshaped = rho.reshape(dim_A, dim_B, dim_A, dim_B)
    rho_pt = rho_reshaped.permute(0, 3, 2, 1).reshape(dim_A * dim_B, dim_A * dim_B)
    return rho_pt



def quantum_loss(predicted, U1, U2, Usep, finalstate, ent_weight=0.5):
    Y, Z = predicted
    target0 = torch.zeros(256, 16, dtype=torch.cfloat, device=Y.device)
    target0[:, 0] = 1
    target1 = torch.zeros(256, 16, dtype=torch.cfloat, device=Z.device)
    target1[:, 15] = 1

    # Fidelity losses
    loss0 = torch.norm(Y - target0, p='fro')
    loss1 = torch.norm(Z - target1, p='fro')
    
    # Unitary constraints
    uni_loss = 0.0
    for U, dim in [(U1, 256), (U2, 256), (Usep, 16)]:
        identity = torch.eye(dim, dtype=torch.cfloat, device=U.device)
        uni_loss += torch.norm(U @ U.conj().T - identity, p='fro')
        uni_loss += torch.norm(U.conj().T @ U - identity, p='fro')
    ent_loss = enhanced_entanglement_loss(finalstate)
    entanglementlossg = enhanced_entanglement_lossnew(Y.reshape(4096))
    entanglementlossh = enhanced_entanglement_lossnew(Z.reshape(4096))
    total_loss = (loss0 + loss1) + 0.1 * uni_loss + ent_weight * ent_loss + ent_weight * entanglementlossg + ent_weight * entanglementlossh
    return total_loss, loss0, loss1, uni_loss, ent_loss,entanglementlossg,entanglementlossh


class SmartScheduler:
    def __init__(self, optimizer, factor=0.5, patience=3, threshold=1e-5):
        self.optimizer = optimizer
        self.factor = factor
        self.patience = patience
        self.threshold = threshold
        self.best_loss = float('inf')
        self.stale_epochs = 0

    def step(self, current_loss):
        if current_loss < self.best_loss - self.threshold:
            self.best_loss = current_loss
            self.stale_epochs = 0
        else:
            self.stale_epochs += 1
            if self.stale_epochs >= self.patience:
                self._adjust_lr()
                self.stale_epochs = 0

    def _adjust_lr(self):
        for param_group in self.optimizer.param_groups:
            param_group['lr'] *= self.factor
            print(f"Learning rate reduced to {param_group['lr']:.2e}")

# Training setup
model = QuantumUnitary()
optimizer = optim.AdamW(model.parameters(), lr=0.1, weight_decay=1e-5)
scheduler = SmartScheduler(optimizer)
epochs = 500

# Training loop
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(model.finalstate)
    total_loss, loss0, loss1, uni_loss, ent_loss,entanglementlossg,entanglementlossh = quantum_loss(
        output, model.U1, model.U2, model.Useparating, model.finalstate
    )
    total_loss.backward()
    optimizer.step()
    scheduler.step(total_loss.item())

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:4d} | Total: {total_loss.item():.6f} | "
              f"Fidelity: {loss0.item()+loss1.item():.6f} | "
              f"Unitary: {uni_loss.item():.6f} | "
              f"Entanglement: {ent_loss.item():.6f} | "
              f"Entanglementg: {entanglementlossg.item():.6f} | "
              f"Entanglementh: {entanglementlossh.item():.6f} | ")

with torch.no_grad():
    final_state = model.finalstate
    state_matrix = final_state.reshape(16, 16)
    _, S, _ = torch.linalg.svd(state_matrix)
    print(f"\nFinal Schmidt rank: {(S > 1e-6).sum().item()}")
    print(f"Final state entanglement loss: {enhanced_entanglement_lossnew(final_state).item():.4f}")

# Rerun the code if the Fidelity is greater than 25, this value varies for each run , but once we have gotten the least Fidelity of all , we can always store the 
# unitary and the other results and can reproduce them. 
# Also Code modification by Kaimming Initialization can be potential way to avoid this ...


Schmidt Rank: 16.0, Entropy: 3.2846670150756836, Reduced Purity: 0.12283187359571457, Negativity: 5.301482677459717
Schmidt Rank: 16.0, Entropy: 3.3250179290771484, Reduced Purity: 0.11838807910680771, Negativity: 5.412406921386719
Schmidt Rank: 16.0, Entropy: 3.335437059402466, Reduced Purity: 0.11706830561161041, Negativity: 5.435468673706055
Schmidt Rank: 16.0, Entropy: 2.5820088386535645, Reduced Purity: 0.2521948218345642, Negativity: 3.925964593887329
Schmidt Rank: 16.0, Entropy: 2.7951807975769043, Reduced Purity: 0.20949605107307434, Negativity: 4.3152031898498535
Schmidt Rank: 16.0, Entropy: 2.773871898651123, Reduced Purity: 0.2143976241350174, Negativity: 4.27772331237793
Schmidt Rank: 16.0, Entropy: 3.2332162857055664, Reduced Purity: 0.14349037408828735, Negativity: 5.3585944175720215
Schmidt Rank: 16.0, Entropy: 3.3257994651794434, Reduced Purity: 0.12994059920310974, Negativity: 5.565574645996094
Schmidt Rank: 16.0, Entropy: 3.3281149864196777, Reduced Purity: 0.12940163

KeyboardInterrupt: 

In [ ]:
# Code to Verify the Output Accuracy of the Unitary of Classification at Bob , Class0 and Class 1 target states.

import numpy as np

model.eval()


with torch.no_grad():
    g, h = model(model.finalstate)
gh = torch.cat([g,h],dim=0)

g_np = g.numpy()
h_np = h.numpy()

data = gh.detach().numpy()

indices_list = []

for vector in data:
    sorted_indices = np.argsort(vector)[-1:]  
    indices_list.append(sorted_indices)


result = [index for sublist in indices_list for index in sublist]


print(len(result)) 


new_result = [round(index / 16) for index in result]


print(new_result)


actual_labels =  np.array([0]*256 + [1]*256)  #(Actual Labels of the target States)
predicted_labels = np.array(new_result)  

# Count the number of matches
matching_counts = np.sum(actual_labels == predicted_labels)
print("Classification Accuracy of Bob's Unitary:", matching_counts) # Classification Accuracy can go to upto 512 i.e.100 percent , if one have multiple reruns of thecode
# or tweak the intilaiziation and the entanglement cosntraints to have weaker cosntraints but still entangled.

512
[0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [7]:
# Code as Entanglement Check for the 8 Qubit State that Alice and BOb Shares.

import torch
from itertools import combinations

class EntanglementVerifier:
    def __init__(self):
        self.state_dim = 256
        self.qubit_count = 8
        
    def compute_reduced_density(self, state, kept_qubits):
        """Calculate reduced density matrix for specified qubits"""
        state_tensor = state.reshape([2]*self.qubit_count)
        kept = sorted(kept_qubits)
        trace_dims = sorted(set(range(self.qubit_count)) - set(kept))
        
        permuted = state_tensor.permute(kept + trace_dims)
        d_keep = 2**len(kept)
        return permuted.reshape(d_keep, -1) @ permuted.reshape(d_keep, -1).conj().T

    def schmidt_decomposition(self, state):
        """Check entanglement via Schmidt rank between 4+4 qubit partition"""
        
        state_matrix = state.reshape(16, 16)  # Split into two 4-qubit systems
        _, S, _ = torch.linalg.svd(state_matrix)
        print(S)
        return (S > 1e-6).sum().item() > 1  # Entangled if Schmidt rank > 1

    def verify(self, state):
        """Comprehensive entanglement verification with multiple criteria"""
        if len(state) != self.state_dim:
            raise ValueError(f"State must be {self.state_dim}-dimensional")
            
        state = state / torch.norm(state)
        print("Entanglement Verification Report\n" + "="*40)
        
        # Method 1: Subsystem entropy analysis
        max_entropy = 0
        for qubits in combinations(range(8), 4):
            rho = self.compute_reduced_density(state, qubits)
            eigvals = torch.linalg.eigvalsh(rho).real.clamp(min=1e-12)
            entropy = -torch.sum(eigvals * torch.log2(eigvals))
            max_entropy = max(max_entropy, entropy)

        # Method 2: Schmidt decomposition check
        schmidt_entangled = self.schmidt_decomposition(state)

        print("="*40)
        if max_entropy > 3.9 or schmidt_entangled:
            print("Entanglement detected")
            if schmidt_entangled:
                print(" Schmidt decomposition confirms entanglement")
                rho = torch.outer(state, state.conj().T)  # Full density matrix
                rho_reduced = partial_trace(rho, 16, 16)  # Partial trace over second subsystem
                A,B,C = torch.linalg.svd(rho_reduced)
                print('Eigen Values of the Reduced Density Matrix',B) 
                reduced_purity = torch.trace(rho_reduced @ rho_reduced).real
                purity_loss = reduced_purity  # Minimize purity of reduced state (more entangled = lower purity)
                print('Purity Loss',reduced_purity)
                print('Entanglement Entropy',max_entropy)

            if max_entropy > 3.9:
                print(f"High subsystem entropy ({max_entropy:.2f}/4.0)")
        else:
            print("No significant entanglement detected")
    
    def compute_reduced_densitytrace(self, state, kept_qubits=8):
        """Efficient reduced density matrix calculation with stabilization"""
        state_tensor = state.reshape([2]*8)
        kept = sorted(kept_qubits)
        trace_dims = sorted(set(range(8)) - set(kept))
        
        permuted = state_tensor.permute(kept + trace_dims)
        d_keep = 2**len(kept)
        reshaped = permuted.reshape(d_keep, -1)
        rho = reshaped @ reshaped.conj().T
        purity = torch.trace(rho@rho).real

        print('Purity',purity)
        return purity

if __name__ == "__main__":
    test_state = model.finalstate
    test_state = test_state / torch.norm(test_state)
    
    verifier = EntanglementVerifier()
    result = verifier.verify(test_state)
    
    if result is not None:
        print(f"Constructed separable state shape: {result.shape}")
        print(f"First 10 elements: {result[:10].numpy()}")

Entanglement Verification Report
tensor([8.2521e-01, 5.2766e-01, 1.8183e-01, 7.2665e-02, 4.1870e-02, 1.8495e-02,
        8.7662e-03, 6.5740e-03, 4.3624e-03, 3.8752e-03, 2.3484e-03, 1.9138e-03,
        1.2624e-03, 9.3284e-04, 3.5938e-04, 1.1475e-04],
       grad_fn=<LinalgSvdBackward0>)
Entanglement detected
 Schmidt decomposition confirms entanglement
Eigen Values of the Reduced Density Matrix tensor([6.8098e-01, 2.7842e-01, 3.3063e-02, 5.2802e-03, 1.7531e-03, 3.4206e-04,
        7.6845e-05, 4.3219e-05, 1.9030e-05, 1.5018e-05, 5.5150e-06, 3.6625e-06,
        1.5940e-06, 8.7045e-07, 1.2928e-07, 1.3262e-08],
       grad_fn=<LinalgSvdBackward0>)
Purity Loss tensor(0.5424, grad_fn=<SelectBackward0>)
Entanglement Entropy tensor(2.1616, grad_fn=<NegBackward0>)


In [ ]:
# Now changing number of qubits of the Architecture by Induction from the Standard Teleporation Protocol and also different criterias of entanglement is the
# Future Scope of this work...

# This is just one of the works going on at Heisenberg Quantum AI and many such interesting projects are going on pararelly.

All Rights Reserved @Heisenberg Quantum AI